# Assignment 2

Data Wrangling II
Create an “Academic performance” dataset of students and perform the
following operations using Python. 

1. Scan all variables for missing values and inconsistencies. 
If there are missing values and/or inconsistencies, use any of the suitable techniques to deal with them.

2. Scan all numeric variables for outliers. 
If there are outliers, use any of the suitable techniques to deal with them.

3. Apply data transformations on at least one of the variables. 
The purpose of this transformation should be one of the following reasons: 
- to change the scale for better understanding of the variable
- to convert a non-linear relation into a linear one
- to decrease the skewness and convert the distribution into a normal distribution.

Reason and document your approach properly.

In [331]:
import pandas as pd

df = pd.read_csv('dataset.csv')
df.head()

,gender,race/ethnicity,parental_level_of_edu,lunch,test_preparation_course,math_score,reading_score,writing_score
0,Female,Group D,high school,standard,completed,NaN,55.0,21.0
1,NaN,Group D,bachelor's degree,Free,completed,62.0,73.0,33.0
2,NaN,group e,some college,standard,completed,25.0,19.0,24.0
3,female,group c,high school,Standard,completed,82.0,32.0,56.0
4,male,group e,high school,standard,completed,87.0,51.0,20.0


In [332]:
df = df.rename(columns={
    "race/ethnicity" : "race", 
    "parental_level_of_edu" : "parent_edu",
    "lunch" : "lunch_type",
    "test_preparation_course" : "course"
    })

In [333]:
df.isna().sum()

gender           107
race              92
parent_edu        80
lunch_type        94
course           105
math_score       105
reading_score    106
writing_score    103
dtype: int64

In [334]:
df["gender"] = df["gender"].str.lower()
df["gender"] = df["gender"].replace({
    "m" : "male",
    "f" : "female"
})
df["gender"] = df["gender"].ffill()
df["gender"] = df["gender"].str.capitalize()
df.gender.unique()

array(['Female', 'Male'], dtype=object)

In [335]:
df["race"] = df["race"].str.upper()
df["race"] = df["race"].ffill()
df["race"] = (df["race"].str.split("GROUP")).str[1]
df["race"].unique()

array([' D', ' E', ' C', ' A', ' B'], dtype=object)

In [336]:
df = df.dropna(subset=["parent_edu"])
df["parent_edu"] = df["parent_edu"].str.lower()
df.loc[df["parent_edu"].str.contains("school",  na=False),"parent_edu"] = "High School"
df.loc[df["parent_edu"].str.contains("master",  na=False),"parent_edu"] = "Master"
df.loc[df["parent_edu"].str.contains("bachelor",  na=False),"parent_edu"] = "Bachelor"
df.loc[df["parent_edu"].str.contains("college",  na=False),"parent_edu"] = "College"
df.loc[df["parent_edu"].str.contains("associate",  na=False), "parent_edu"] = "Associate"

df.parent_edu.unique()

array(['High School', 'Bachelor', 'College', 'Master', 'Associate'],
      dtype=object)

In [337]:
df.lunch_type.unique()

df.lunch_type = df.lunch_type.ffill()
df.lunch_type = df.lunch_type.str.lower()
df.loc[df["lunch_type"].str.contains("free"), "lunch_type"] = "free"
df.lunch_type = df.lunch_type.str.capitalize()
df.lunch_type.unique()

array(['Standard', 'Free'], dtype=object)

In [338]:
df["course"] = df["course"].fillna("none").ne("none")
df["course"].unique


<bound method Series.unique of 0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: course, Length: 920, dtype: bool>

In [339]:
df.isna().sum()

gender            0
race              0
parent_edu        0
lunch_type        0
course            0
math_score       97
reading_score    97
writing_score    92
dtype: int64

In [340]:
df["math_score"] = df["math_score"].fillna(df["math_score"].mean())
df["reading_score"] = df["reading_score"].fillna(df["reading_score"].mean())
df["writing_score"] = df["writing_score"].fillna(df["writing_score"].mean()) 

df.isna().sum()

gender           0
race             0
parent_edu       0
lunch_type       0
course           0
math_score       0
reading_score    0
writing_score    0
dtype: int64

In [341]:
# Min Max Normalization
def min_max_normalization(df, col):
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df[col]
df.head()


,gender,race,parent_edu,lunch_type,course,math_score,reading_score,writing_score
0,Female,D,High School,Standard,True,58.235723,55.0,21.0
1,Female,D,Bachelor,Free,True,62.000000,73.0,33.0
2,Female,E,College,Standard,True,25.000000,19.0,24.0
3,Female,C,High School,Standard,True,82.000000,32.0,56.0
4,Male,E,High School,Standard,True,87.000000,51.0,20.0


In [342]:
min_max_normalized_df = df.copy()
min_max_normalized_df["math_score"] = min_max_normalization(min_max_normalized_df, "math_score")
min_max_normalized_df["reading_score"] = min_max_normalization(min_max_normalized_df, "reading_score")
min_max_normalized_df["writing_score"] = min_max_normalization(min_max_normalized_df, "writing_score")
min_max_normalized_df.head()

,gender,race,parent_edu,lunch_type,course,math_score,reading_score,writing_score
0,Female,D,High School,Standard,True,0.511319,0.487805,0.111111
1,Female,D,Bachelor,Free,True,0.569231,0.707317,0.277778
2,Female,E,College,Standard,True,0.000000,0.048780,0.152778
3,Female,C,High School,Standard,True,0.876923,0.207317,0.597222
4,Male,E,High School,Standard,True,0.953846,0.439024,0.097222
